# `Exploratory Data Analysis`

This document constitute the first part of the analysis for Sign Language Recognition from Google dataset. 
We will Explore first the data throw Exploratory Data Analysis, Visuals our data by creating some animated videos from the positional data, Examine further the data with the occurences of certain patterns throw Dictionary Frequency Analysis, and then Follow up with Preprocessing before going to modeling stage. (Where we will see later the PreNorm Transformer architecture)

### The preparation of the data is divided in 4 parts:

## *`Exploratory Data Analysis`* -> This notebook

### `Visuals`

### `Dictionary Frequency Analysis`

### `Preprocessing`



``FINDINGS :
``

based on the entire file of 160 000 rows(frames):

- Percentage of missing left hand  values: ``90.10%``

- Percentage of missing right hand values: ``50.50%``

- Percentage of rows with missing values for both left and right hand values for an identical frame : ``41.02%``

- The pose landmark seem to be calibrated differently than hand : they have more negative values, and values higher than 1

- When reduced to our features of interest (without face features), the file is reduced to a memory usage: ``138.6 MB`` for our total of 226 features : sequence_id, frame, 63 righthand,63 lefthand, 99 pose (ps all of the parquet file are about the same weight). We will later decide to keep only certain specific pose landmark such as arms reducing by about 20*3 landmark points.
-> Some of the missing data can be removed directly, including the case where we have no values of the hand at the beginning or the end of a clip

- There is exactly 1000 clip on the file. It is likely that the parquet file have been divided in 1000 clips each.

- The average number of frame is 160 per clip. The maximum being 488. (based on the maximum number of frame from each sequence_id +1 as we start from frame 0 ; the mean of the column would be unappropriate in this case)

Import necessaries libraries :

In [7]:
import pyarrow.parquet as pq
import pandas as pd

### *Let's Define below the name of the variant df we will be creating*

### Index of the df names : 

df : df of the entire parquet file unfiltered

cdf : as Columns of interests of our df : sequence_id, frame, (xyz)right_hand_(0 to 20), (xyz)left_hand_(0 to 20), (xyz)pose_(0 to 33)

reduced_df : as Reduced df with only 40107 rows of cdf if ever need to try some operations even faster (40 107 as wanted to have 40 000 and not to cut the sequence of frame)

#### ``test_df ``: as test df for a good understanding of missing values pattern from the entire parquet file with sequence_id, frame, 1 left hand landmark, 1 right hand landmark (We identified that when there is missing value for a hand, all of its landmark are missing ; there is no 'pose' landmark missing)



Let's have an overview of the data unfiltered (chose a random parquet file that we will keep all along ``'527708222'`` , i've open 3 parquet of the 68 total in the train set, they were similar in term of average/repartition frame per clip, number of rows, same features and therefore seem that clip are random on each parquet and this belief has been consolidated on this analysis)

In [16]:

# Replace 'path_to_file.parquet' with the actual file path on your system
# For the rest of this EDA we will focus on this file combining 1000 sequences. 
file_path = 'd:/Bureau/DS/aCapstone/527708222.parquet'

# Open the Parquet file using ParquetFile
parquet_file = pq.ParquetFile(file_path)

row_group_index = 0

# Read the specific row group from the Parquet file
row_group_table = parquet_file.read_row_group(row_group_index)

df = row_group_table.to_pandas()

df.head(10)

,frame,x_face_0,x_face_1,x_face_2,x_face_3,x_face_4,x_face_5,x_face_6,x_face_7,x_face_8,...,z_right_hand_11,z_right_hand_12,z_right_hand_13,z_right_hand_14,z_right_hand_15,z_right_hand_16,z_right_hand_17,z_right_hand_18,z_right_hand_19,z_right_hand_20
sequence_id,,,,,,,,,,,,,,,,,,,,,
1880607618,0,0.487255,0.492607,0.494152,0.480811,0.492845,0.494270,0.498394,0.368207,0.500288,...,-0.196956,-0.211209,-0.142313,-0.193477,-0.205450,-0.208683,-0.175401,-0.209275,-0.210895,-0.208162
1880607618,1,0.488399,0.492968,0.494502,0.480484,0.493159,0.494415,0.497935,0.367207,0.499401,...,-0.171502,-0.190701,-0.119451,-0.172471,-0.190162,-0.196562,-0.155715,-0.194636,-0.196822,-0.193751
1880607618,2,0.491521,0.493348,0.495074,0.480274,0.493283,0.494308,0.497388,0.367744,0.498564,...,-0.154713,-0.179518,-0.115403,-0.165406,-0.195193,-0.215218,-0.156990,-0.203963,-0.222674,-0.234745
1880607618,3,0.486116,0.488677,0.490791,0.475851,0.488651,0.489896,0.493834,0.366394,0.495337,...,-0.173814,-0.202558,-0.130080,-0.186153,-0.220297,-0.244129,-0.178186,-0.233190,-0.258209,-0.275004
1880607618,4,0.492289,0.495260,0.498130,0.480569,0.494592,0.494893,0.496912,0.366336,0.497359,...,-0.117347,-0.142112,-0.075403,-0.122787,-0.162759,-0.189201,-0.105377,-0.156895,-0.184629,-0.201269
1880607618,5,0.509225,0.505830,0.507423,0.489369,0.505046,0.504673,0.504471,0.366964,0.503673,...,-0.106575,-0.134838,-0.068661,-0.107393,-0.142445,-0.167589,-0.102564,-0.144277,-0.165639,-0.179552
1880607618,6,0.502408,0.504786,0.506150,0.489051,0.504134,0.503930,0.504243,0.370887,0.503841,...,-0.068458,-0.094906,-0.062629,-0.095716,-0.131119,-0.158647,-0.109830,-0.142800,-0.157586,-0.170328
1880607618,7,0.513588,0.497824,0.499674,0.483226,0.497445,0.497878,0.499962,0.372318,0.500624,...,-0.020596,-0.034042,-0.037462,-0.073540,-0.097971,-0.112151,-0.088628,-0.126112,-0.132545,-0.133995
1880607618,8,0.502060,0.494293,0.496561,0.480990,0.494168,0.495263,0.498975,0.373701,0.500522,...,-0.076957,-0.088459,-0.052694,-0.104890,-0.119799,-0.118821,-0.091256,-0.131648,-0.132252,-0.124047


In [18]:
# Get the names of all columns
all_column_names = df.columns.tolist()

# Print all 1600 column names
print(all_column_names)

['frame', 'x_face_0', 'x_face_1', 'x_face_2', 'x_face_3', 'x_face_4', 'x_face_5', 'x_face_6', 'x_face_7', 'x_face_8', 'x_face_9', 'x_face_10', 'x_face_11', 'x_face_12', 'x_face_13', 'x_face_14', 'x_face_15', 'x_face_16', 'x_face_17', 'x_face_18', 'x_face_19', 'x_face_20', 'x_face_21', 'x_face_22', 'x_face_23', 'x_face_24', 'x_face_25', 'x_face_26', 'x_face_27', 'x_face_28', 'x_face_29', 'x_face_30', 'x_face_31', 'x_face_32', 'x_face_33', 'x_face_34', 'x_face_35', 'x_face_36', 'x_face_37', 'x_face_38', 'x_face_39', 'x_face_40', 'x_face_41', 'x_face_42', 'x_face_43', 'x_face_44', 'x_face_45', 'x_face_46', 'x_face_47', 'x_face_48', 'x_face_49', 'x_face_50', 'x_face_51', 'x_face_52', 'x_face_53', 'x_face_54', 'x_face_55', 'x_face_56', 'x_face_57', 'x_face_58', 'x_face_59', 'x_face_60', 'x_face_61', 'x_face_62', 'x_face_63', 'x_face_64', 'x_face_65', 'x_face_66', 'x_face_67', 'x_face_68', 'x_face_69', 'x_face_70', 'x_face_71', 'x_face_72', 'x_face_73', 'x_face_74', 'x_face_75', 'x_face_76',

We can observe and confirm that the data are : 
- the sequence_id that we will use with the landmark file to determine what this clip label is
- the frame of the clip
- the 4 features being 468 * 3 face, 21 * 3 left hand, 21 * 3 right hand, 33 * 3 pose 'where 3 is xyz dimensions'; We have determine that the face landmark won't be useful for us in alphabet ASL and will largely speed up the process

In [26]:
#For later : 
#print("TensorFlow v" + tf.__version__)
#print("Mediapipe v" + mediapipe.__version__)

We will below select the landmark interesting in our case. Removing Face landmarks and potentially removing pose landmarks.

In [29]:
columns_of_interest = ['sequence_id',
'frame',
'x_right_hand_0', 'x_right_hand_1', 'x_right_hand_2', 'x_right_hand_3', 'x_right_hand_4', 'x_right_hand_5', 'x_right_hand_6', 'x_right_hand_7', 'x_right_hand_8', 'x_right_hand_9', 'x_right_hand_10', 'x_right_hand_11', 'x_right_hand_12', 'x_right_hand_13', 'x_right_hand_14', 'x_right_hand_15', 'x_right_hand_16', 'x_right_hand_17', 'x_right_hand_18', 'x_right_hand_19', 'x_right_hand_20',
'y_right_hand_0', 'y_right_hand_1', 'y_right_hand_2', 'y_right_hand_3', 'y_right_hand_4', 'y_right_hand_5', 'y_right_hand_6', 'y_right_hand_7', 'y_right_hand_8', 'y_right_hand_9', 'y_right_hand_10', 'y_right_hand_11', 'y_right_hand_12', 'y_right_hand_13', 'y_right_hand_14', 'y_right_hand_15', 'y_right_hand_16', 'y_right_hand_17', 'y_right_hand_18', 'y_right_hand_19', 'y_right_hand_20',
'z_right_hand_0', 'z_right_hand_1', 'z_right_hand_2', 'z_right_hand_3', 'z_right_hand_4', 'z_right_hand_5', 'z_right_hand_6', 'z_right_hand_7', 'z_right_hand_8', 'z_right_hand_9', 'z_right_hand_10', 'z_right_hand_11', 'z_right_hand_12', 'z_right_hand_13', 'z_right_hand_14', 'z_right_hand_15', 'z_right_hand_16', 'z_right_hand_17', 'z_right_hand_18', 'z_right_hand_19', 'z_right_hand_20',
'x_left_hand_0', 'x_left_hand_1', 'x_left_hand_2', 'x_left_hand_3', 'x_left_hand_4', 'x_left_hand_5', 'x_left_hand_6', 'x_left_hand_7', 'x_left_hand_8', 'x_left_hand_9', 'x_left_hand_10', 'x_left_hand_11', 'x_left_hand_12', 'x_left_hand_13', 'x_left_hand_14', 'x_left_hand_15', 'x_left_hand_16', 'x_left_hand_17', 'x_left_hand_18', 'x_left_hand_19', 'x_left_hand_20',
'y_left_hand_0', 'y_left_hand_1', 'y_left_hand_2', 'y_left_hand_3', 'y_left_hand_4', 'y_left_hand_5', 'y_left_hand_6', 'y_left_hand_7', 'y_left_hand_8', 'y_left_hand_9', 'y_left_hand_10', 'y_left_hand_11', 'y_left_hand_12', 'y_left_hand_13', 'y_left_hand_14', 'y_left_hand_15', 'y_left_hand_16', 'y_left_hand_17', 'y_left_hand_18', 'y_left_hand_19', 'y_left_hand_20',
'z_left_hand_0', 'z_left_hand_1', 'z_left_hand_2', 'z_left_hand_3', 'z_left_hand_4', 'z_left_hand_5', 'z_left_hand_6', 'z_left_hand_7', 'z_left_hand_8', 'z_left_hand_9', 'z_left_hand_10', 'z_left_hand_11', 'z_left_hand_12', 'z_left_hand_13', 'z_left_hand_14', 'z_left_hand_15', 'z_left_hand_16', 'z_left_hand_17', 'z_left_hand_18', 'z_left_hand_19', 'z_left_hand_20',
'x_pose_0', 'x_pose_1', 'x_pose_2', 'x_pose_3', 'x_pose_4', 'x_pose_5', 'x_pose_6', 'x_pose_7', 'x_pose_8', 'x_pose_9', 'x_pose_10', 'x_pose_11', 'x_pose_12', 'x_pose_13', 'x_pose_14', 'x_pose_15', 'x_pose_16', 'x_pose_17', 'x_pose_18', 'x_pose_19', 'x_pose_20', 'x_pose_21', 'x_pose_22', 'x_pose_23', 'x_pose_24', 'x_pose_25', 'x_pose_26', 'x_pose_27', 'x_pose_28', 'x_pose_29', 'x_pose_30', 'x_pose_31', 'x_pose_32',
'y_pose_0', 'y_pose_1', 'y_pose_2', 'y_pose_3', 'y_pose_4', 'y_pose_5', 'y_pose_6', 'y_pose_7', 'y_pose_8', 'y_pose_9', 'y_pose_10', 'y_pose_11', 'y_pose_12', 'y_pose_13', 'y_pose_14', 'y_pose_15', 'y_pose_16', 'y_pose_17', 'y_pose_18', 'y_pose_19', 'y_pose_20', 'y_pose_21', 'y_pose_22', 'y_pose_23', 'y_pose_24', 'y_pose_25', 'y_pose_26', 'y_pose_27', 'y_pose_28', 'y_pose_29', 'y_pose_30', 'y_pose_31', 'y_pose_32',
'z_pose_0', 'z_pose_1', 'z_pose_2', 'z_pose_3', 'z_pose_4', 'z_pose_5', 'z_pose_6', 'z_pose_7', 'z_pose_8', 'z_pose_9', 'z_pose_10', 'z_pose_11', 'z_pose_12', 'z_pose_13', 'z_pose_14', 'z_pose_15', 'z_pose_16', 'z_pose_17', 'z_pose_18', 'z_pose_19', 'z_pose_20', 'z_pose_21', 'z_pose_22', 'z_pose_23', 'z_pose_24', 'z_pose_25', 'z_pose_26', 'z_pose_27', 'z_pose_28', 'z_pose_29', 'z_pose_30', 'z_pose_31', 'z_pose_32']

table = pq.read_table('d:/Bureau/DS/aCapstone/527708222.parquet', columns=columns_of_interest)
cdf = table.to_pandas()
cdf

,frame,x_right_hand_0,x_right_hand_1,x_right_hand_2,x_right_hand_3,x_right_hand_4,x_right_hand_5,x_right_hand_6,x_right_hand_7,x_right_hand_8,...,z_pose_23,z_pose_24,z_pose_25,z_pose_26,z_pose_27,z_pose_28,z_pose_29,z_pose_30,z_pose_31,z_pose_32
sequence_id,,,,,,,,,,,,,,,,,,,,,
1880607618,0,0.200624,0.302635,0.427209,0.531130,0.609704,0.466830,0.621508,0.720132,0.782646,...,-0.162131,0.168272,-0.037486,0.404005,1.276582,1.570643,1.318286,1.633371,0.062651,0.314512
1880607618,1,0.240830,0.374630,0.474594,0.547350,0.596090,0.453041,0.591468,0.683067,0.740463,...,-0.136024,0.141433,0.041872,0.344083,1.340694,1.408339,1.390278,1.482522,0.192373,0.216677
1880607618,2,0.235014,0.375686,0.485869,0.566487,0.630853,0.434785,0.536943,0.621403,0.682213,...,-0.141135,0.146788,-0.109030,0.310549,1.180241,1.382393,1.227441,1.447687,0.013639,0.167156
1880607618,3,0.252615,0.385305,0.487498,0.563021,0.643407,0.445335,0.525891,0.612506,0.690043,...,-0.146302,0.151999,-0.119314,0.254604,1.178714,1.224841,1.227554,1.292624,0.024260,0.009302
1880607618,4,0.294039,0.405612,0.502041,0.568720,0.631543,0.479026,0.542870,0.605044,0.662126,...,-0.153527,0.159551,-0.227373,0.150963,0.930902,0.949471,0.968373,0.998937,-0.236796,-0.257720
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1912627838,89,0.266774,0.357957,0.415935,0.442117,0.414395,0.271646,0.259206,0.337859,0.368670,...,-0.145449,0.151864,0.071900,0.934476,1.528550,2.599374,1.597583,2.742445,0.393970,1.460342
1912627838,90,0.244922,0.342586,0.392606,0.409731,0.378050,0.240688,0.278423,0.343222,0.337006,...,-0.143819,0.150366,0.095805,0.853011,1.627043,2.312991,1.700490,2.438057,0.443149,1.234005
1912627838,91,0.230954,0.339997,0.389223,0.398273,0.359395,0.228129,0.315546,0.352227,0.322500,...,-0.147698,0.154105,0.093885,0.748713,1.616127,2.175098,1.686710,2.286521,0.455095,0.951585


In [44]:
cdf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 159737 entries, 1880607618 to 1912627838
Columns: 226 entries, frame to z_pose_32
dtypes: float32(225), int16(1)
memory usage: 138.6 MB


In [4]:
# Specify the number of rows you want to keep (in this case, 30,000)
num_rows_to_keep = 40107

# Use DataFrame slicing to select only the first 30,000 rows
reduced_df = cdf[:num_rows_to_keep]

# Now 'reduced_df' contains only the first 30,000 rows of the original DataFrame 'df'
reduced_df.tail(10)

,frame,x_right_hand_0,x_right_hand_1,x_right_hand_2,x_right_hand_3,x_right_hand_4,x_right_hand_5,x_right_hand_6,x_right_hand_7,x_right_hand_8,...,z_pose_23,z_pose_24,z_pose_25,z_pose_26,z_pose_27,z_pose_28,z_pose_29,z_pose_30,z_pose_31,z_pose_32
sequence_id,,,,,,,,,,,,,,,,,,,,,
1889194597,237,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.212467,0.219760,-0.216419,0.554647,0.941297,1.600518,0.985774,1.678939,-0.122747,0.513253
1889194597,238,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.202671,0.209900,-0.221928,0.596431,0.876993,1.675096,0.919642,1.754117,-0.272520,0.538907
1889194597,239,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.206368,0.213502,-0.257335,0.668806,0.686410,1.706912,0.700564,1.783051,-0.535058,0.590749
1889194597,240,0.211145,0.299697,0.377033,0.416756,0.390703,0.300828,0.292533,0.331550,0.362589,...,-0.182412,0.189879,-0.236748,0.601819,0.746618,1.639129,0.761929,1.718554,-0.500284,0.545904
1889194597,241,0.259661,0.346610,0.419217,0.452936,0.456187,0.341913,0.343807,0.387341,0.433697,...,-0.226689,0.233686,-0.194264,0.849061,0.938508,2.171255,0.966238,2.281338,-0.322804,1.054094
1889194597,242,0.293540,0.380188,0.445382,0.480519,0.521557,0.387373,0.393184,0.442771,0.486210,...,-0.226418,0.233340,-0.063977,0.719934,1.163040,1.980953,1.204851,2.082672,-0.032002,0.843896
1889194597,243,0.324132,0.404965,0.470855,0.508212,0.557369,0.426497,0.430681,0.484099,0.529180,...,-0.217401,0.224046,-0.104506,0.419008,1.043961,1.390895,1.079365,1.460293,-0.114810,0.224111
1889194597,244,0.349544,0.429875,0.492972,0.527298,0.574678,0.456873,0.473039,0.526615,0.573421,...,-0.202579,0.208712,-0.065932,0.276495,1.123341,1.211558,1.166461,1.277241,0.016934,0.071297
1889194597,245,0.363576,0.447008,0.505395,0.542994,0.594771,0.461990,0.487438,0.554616,0.605582,...,-0.217099,0.222818,-0.085839,0.313904,1.050791,1.250910,1.088495,1.317916,-0.074060,0.120400


## I've made some test and verification, and observed that if there is ever a missing data for 'x_right_hand', than all right hand data is missing, and same for left hand. Therefore to simplify and speed the EDA on this part i will make a specific df 'tdf' for testdf to analyze further the position and patterns of missing data.

### `below will be the small df really useful to have answers easier to visualize regarding the missing data`

In [8]:
#if we are looking for missing value we have explored before that the number of the landmark doesn't make any difference.
columns_of_interest = ['sequence_id', 'frame','x_right_hand_0' , 'x_left_hand_0', 'x_pose_0' ]


table = pq.read_table('d:/Bureau/DS/aCapstone/527708222.parquet', columns=columns_of_interest)
test_df = table.to_pandas()
test_df

,frame,x_right_hand_0,x_left_hand_0,x_pose_0
sequence_id,,,,
1880607618,0,0.200624,NaN,0.515170
1880607618,1,0.240830,NaN,0.515566
1880607618,2,0.235014,NaN,0.515175
1880607618,3,0.252615,NaN,0.515144
1880607618,4,0.294039,NaN,0.514626
...,...,...,...,...
1912627838,89,0.266774,NaN,0.533969
1912627838,90,0.244922,NaN,0.533277
1912627838,91,0.230954,NaN,0.531930


In [10]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 159737 entries, 1880607618 to 1912627838
Data columns (total 4 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   frame           159737 non-null  int16  
 1   x_right_hand_0  79071 non-null   float32
 2   x_left_hand_0   15819 non-null   float32
 3   x_pose_0        159737 non-null  float32
dtypes: float32(3), int16(1)
memory usage: 3.4 MB


Let's look at few hundreds of rows for right and left hand to have some understanding of the most common patterns. 

In [9]:
rows = test_df.iloc[1050:1100]
rows

,frame,x_right_hand_0,x_left_hand_0,x_pose_0
sequence_id,,,,
1880710522,239,0.261832,NaN,0.762327
1880710522,240,0.263437,NaN,0.761998
1880710522,241,0.268146,NaN,0.761625
1880710522,242,0.266766,NaN,0.761475
1880710522,243,0.264296,NaN,0.762506
1880710522,244,0.262925,NaN,0.764503
1880710522,245,0.269693,NaN,0.765661
1880710522,246,0.277626,NaN,0.766420
1880773667,0,0.124537,NaN,0.479875


In [10]:
reduced_df.tail()

,frame,x_right_hand_0,x_right_hand_1,x_right_hand_2,x_right_hand_3,x_right_hand_4,x_right_hand_5,x_right_hand_6,x_right_hand_7,x_right_hand_8,...,z_pose_23,z_pose_24,z_pose_25,z_pose_26,z_pose_27,z_pose_28,z_pose_29,z_pose_30,z_pose_31,z_pose_32
sequence_id,,,,,,,,,,,,,,,,,,,,,
1889194597,242,0.293540,0.380188,0.445382,0.480519,0.521557,0.387373,0.393184,0.442771,0.486210,...,-0.226418,0.233340,-0.063977,0.719934,1.163040,1.980953,1.204851,2.082672,-0.032002,0.843896
1889194597,243,0.324132,0.404965,0.470855,0.508212,0.557369,0.426497,0.430681,0.484099,0.529180,...,-0.217401,0.224046,-0.104506,0.419008,1.043961,1.390895,1.079365,1.460293,-0.114810,0.224111
1889194597,244,0.349544,0.429875,0.492972,0.527298,0.574678,0.456873,0.473039,0.526615,0.573421,...,-0.202579,0.208712,-0.065932,0.276495,1.123341,1.211558,1.166461,1.277241,0.016934,0.071297
1889194597,245,0.363576,0.447008,0.505395,0.542994,0.594771,0.461990,0.487438,0.554616,0.605582,...,-0.217099,0.222818,-0.085839,0.313904,1.050791,1.250910,1.088495,1.317916,-0.074060,0.120400
1889194597,246,0.376617,0.467129,0.516210,0.557677,0.605394,0.450785,0.521018,0.597762,0.662597,...,-0.190818,0.195910,-0.110302,0.166782,1.018001,0.996880,1.055224,1.052139,-0.096777,-0.137256


In [11]:
# Display information about the DataFrame
print(reduced_df.info()) 


<class 'pandas.core.frame.DataFrame'>
Int64Index: 40107 entries, 1880607618 to 1889194597
Columns: 226 entries, frame to z_pose_32
dtypes: float32(225), int16(1)
memory usage: 34.8 MB
None


In [12]:
print(test_df.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 159737 entries, 1880607618 to 1912627838
Data columns (total 4 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   frame           159737 non-null  int16  
 1   x_right_hand_0  79071 non-null   float32
 2   x_left_hand_0   15819 non-null   float32
 3   x_pose_0        159737 non-null  float32
dtypes: float32(3), int16(1)
memory usage: 3.4 MB
None


In [14]:
test_df['frame'].describe()

count    159737.000000
mean        100.526434
std          77.515498
min           0.000000
25%          41.000000
50%          84.000000
75%         142.000000
max         477.000000
Name: frame, dtype: float64

In [14]:
num_frame_zero = (test_df['frame'] == 0).sum()
print("Number of occurrences where 'frame' is equal to 0:", num_frame_zero)

Number of occurrences where 'frame' is equal to 0: 1000


In [15]:
# To count the number of individual 'sequence_id':
sequence_id_count = test_df.index.nunique()

# To display the count:
print("Number of individual 'sequence_id':", sequence_id_count)

Number of individual 'sequence_id': 1000


There is indeed 1000 individual clip on the entire parquet file. One of the assumption now is that the parquet file have been divided by number of 1000 clips.

In [43]:
df.describe()

,frame,x_face_0,x_face_1,x_face_2,x_face_3,x_face_4,x_face_5,x_face_6,x_face_7,x_face_8,...,z_right_hand_11,z_right_hand_12,z_right_hand_13,z_right_hand_14,z_right_hand_15,z_right_hand_16,z_right_hand_17,z_right_hand_18,z_right_hand_19,z_right_hand_20
count,159737.000000,158079.000000,158079.000000,158079.000000,158079.000000,158079.000000,158079.000000,158079.000000,158079.000000,158079.000000,...,79071.000000,79071.000000,79071.000000,79071.000000,79071.000000,79071.000000,79071.000000,79071.000000,79071.000000,79071.000000
mean,100.526434,0.563034,0.556766,0.560131,0.545142,0.556270,0.557161,0.560540,0.456035,0.561187,...,-0.130040,-0.138822,-0.065295,-0.123681,-0.133699,-0.127837,-0.083726,-0.127713,-0.131378,-0.125190
std,77.515498,0.128096,0.125487,0.126958,0.127104,0.125449,0.126134,0.128533,0.133690,0.129601,...,0.064634,0.070789,0.044368,0.057992,0.062825,0.069787,0.046848,0.057291,0.062557,0.068797
min,0.000000,0.139401,0.139812,0.142262,0.114280,0.137150,0.133039,0.122294,-0.003946,0.113283,...,-0.769656,-0.859035,-0.401512,-0.594859,-0.710160,-0.775224,-0.406312,-0.575031,-0.660658,-0.715704
25%,41.000000,0.503193,0.497401,0.500473,0.485545,0.496700,0.497231,0.499582,0.396208,0.499447,...,-0.158860,-0.170374,-0.085129,-0.151818,-0.163864,-0.161134,-0.105896,-0.156339,-0.162685,-0.159375
50%,84.000000,0.583452,0.574527,0.578947,0.564059,0.574055,0.575168,0.579094,0.480648,0.579747,...,-0.118786,-0.126397,-0.056718,-0.114286,-0.122702,-0.115975,-0.074949,-0.118292,-0.120805,-0.114165
75%,142.000000,0.647753,0.639609,0.644261,0.628990,0.639161,0.640669,0.646546,0.543821,0.648506,...,-0.088001,-0.092064,-0.035320,-0.085261,-0.091183,-0.079632,-0.051505,-0.088429,-0.088077,-0.077192
max,477.000000,1.009768,1.005770,1.010701,0.989611,1.005418,1.006906,1.011670,0.854268,1.013019,...,0.416102,0.443820,0.239639,0.334144,0.366041,0.384823,0.220229,0.296663,0.334309,0.360645


In [ ]:
file_path2 = 'd:/Bureau/DS/aCapstone/527708222.parquet'
# Open the Parquet file using ParquetFile
parquet_file2 = pq.ParquetFile(file_path2)

# Define the row group index you want to read
# For example, to read the first row group, use index 0
row_group_index2 = 0

# Read the specific row group from the Parquet file
row_group_table2 = parquet_file2.read_row_group(row_group_index2)

# Convert the row group to a pandas DataFrame
df2 = row_group_table2.to_pandas()

# Now you have a DataFrame with a part of the data from the Parquet file
df2.head(5)

,frame,x_face_0,x_face_1,x_face_2,x_face_3,x_face_4,x_face_5,x_face_6,x_face_7,x_face_8,...,z_right_hand_11,z_right_hand_12,z_right_hand_13,z_right_hand_14,z_right_hand_15,z_right_hand_16,z_right_hand_17,z_right_hand_18,z_right_hand_19,z_right_hand_20
sequence_id,,,,,,,,,,,,,,,,,,,,,
1880607618,0,0.487255,0.492607,0.494152,0.480811,0.492845,0.494270,0.498394,0.368207,0.500288,...,-0.196956,-0.211209,-0.142313,-0.193477,-0.205450,-0.208683,-0.175401,-0.209275,-0.210895,-0.208162
1880607618,1,0.488399,0.492968,0.494502,0.480484,0.493159,0.494415,0.497935,0.367207,0.499401,...,-0.171502,-0.190701,-0.119451,-0.172471,-0.190162,-0.196562,-0.155715,-0.194636,-0.196822,-0.193751
1880607618,2,0.491521,0.493348,0.495074,0.480274,0.493283,0.494308,0.497388,0.367744,0.498564,...,-0.154713,-0.179518,-0.115403,-0.165406,-0.195193,-0.215218,-0.156990,-0.203963,-0.222674,-0.234745
1880607618,3,0.486116,0.488677,0.490791,0.475851,0.488651,0.489896,0.493834,0.366394,0.495337,...,-0.173814,-0.202558,-0.130080,-0.186153,-0.220297,-0.244129,-0.178186,-0.233190,-0.258209,-0.275004
1880607618,4,0.492289,0.495260,0.498130,0.480569,0.494592,0.494893,0.496912,0.366336,0.497359,...,-0.117347,-0.142112,-0.075403,-0.122787,-0.162759,-0.189201,-0.105377,-0.156895,-0.184629,-0.201269


####  The data should not have any videos or image, but let's confirm :

In [ ]:
#import cv2 
#no need to run if as it takes time, the result is that there is indeed no video in the parquet file
def has_video(parquet_file_path):
    try:
        # Read the Parquet file
        table = pq.read_table(parquet_file_path)

        # Get the first row of the table
        first_row = table[0]

        # Get the first column of the first row
        first_column = first_row[0]

        # Convert the PyArrow array to a NumPy array
        array = first_column.to_numpy()

        # Decode the data using OpenCV
        frame = cv2.imdecode(array, cv2.IMREAD_UNCHANGED)

        # Check if the decoded frame is not empty and is a video frame
        if frame is not None and len(frame.shape) == 3 and frame.shape[2] > 1:
            return True

    except Exception as e:
        # Handle any exceptions (e.g., file not found, invalid Parquet file, etc.)
        print("Error:", e)

    return False

# Example usage
parquet_file_path = "d:/Bureau/DS/aCapstone/175396851.parquet"
if has_video(parquet_file_path):
    print("The Parquet file contains video data.")
else:
    print("The Parquet file does not contain video data.")

Error: 'pyarrow.lib.Int16Scalar' object has no attribute 'to_numpy'
The Parquet file does not contain video data.


#### There is indeed no videos or images in the file. 

In [19]:
test_df['frame'].describe()

count    159737.000000
mean        100.526434
std          77.515498
min           0.000000
25%          41.000000
50%          84.000000
75%         142.000000
max         477.000000
Name: frame, dtype: float64

In [18]:
# Count the number of missing values in the 'x_left_hand_10' column
missing_x_left_hand_10_count = test_df['x_left_hand_0'].isna().sum()

# Print the count of rows where 'x_left_hand_10' has no value
print("Number of left hand with no value:", missing_x_left_hand_10_count)

Number of left hand with no value: 143918


In [ ]:
# Count the number of missing values in the 'x_left_hand_10' column
missing_x_left_hand_0_count = test_df['x_right_hand_0'].isna().sum()

# Print the count of rows where 'x_left_hand_10' has no value
print("Number of left hand with no value:", missing_x_left_hand_10_count)

Number of 'x_left_hand_10' with no value: 21709


In [25]:
# Calculate the counts of missing values in 'x_left_hand_0' and 'x_right_hand_0' columns
missing_x_left_hand_0_count = test_df['x_left_hand_0'].isna().sum()
missing_x_right_hand_0_count = test_df['x_right_hand_0'].isna().sum()

# Calculate the percentages of missing values relative to the total number of rows (159737)
total_rows = 159737
missing_x_left_hand_0_percentage = (missing_x_left_hand_0_count / total_rows) * 100
missing_x_right_hand_0_percentage = (missing_x_right_hand_0_count / total_rows) * 100

# Print the counts of rows where both 'x_left_hand_0' and 'x_right_hand_0' have no value
print("Number of left hand with no value out of 159737 rows:", missing_x_left_hand_0_count)
print("Percentage of missing left hand values: {:.2f}%".format(missing_x_left_hand_0_percentage))
print("Number of right hand with no value out of 159737 rows:", missing_x_right_hand_0_count)
print("Percentage of missing right hand values: {:.2f}%".format(missing_x_right_hand_0_percentage))


Number of left hand with no value out of 159737 rows: 143918
Percentage of missing left hand values: 90.10%
Number of right hand with no value out of 159737 rows: 80666
Percentage of missing right hand values: 50.50%


In [23]:
# Check for missing values in 'x_left_hand_0' and 'x_right_hand_0' columns
is_x_left_hand_0_missing = test_df['x_left_hand_0'].isna()
is_x_right_hand_0_missing = test_df['x_right_hand_0'].isna()

# Combine the conditions using the logical AND operator
# This gives True for rows where both 'x_left_hand_0' and 'x_right_hand_0' have no value
rows_with_both_missing = is_x_left_hand_0_missing & is_x_right_hand_0_missing

# Count the number of rows where both 'x_left_hand_0' and 'x_right_hand_0' have no values
count_both_missing = rows_with_both_missing.sum()

# Print the count of rows where both 'x_left_hand_0' and 'x_right_hand_0' have no values
print("Number of rows with missing values for both left hand and right hand out of 159737 rows:", count_both_missing)

Number of rows with missing values for both left hand and right hand out of 159737 rows: 65524


In [20]:
# Check for missing values in 'x_left_hand_0' and 'x_right_hand_0' columns
both_missing_rows = test_df['x_left_hand_0'].isna() & test_df['x_right_hand_0'].isna()

# Count the number of rows where both 'x_left_hand_0' and 'x_right_hand_0' have no values
count_both_missing = both_missing_rows.sum()

# Calculate the percentage of rows with missing values for both 'x_left_hand_0' and 'x_right_hand_0'
percentage_both_missing = (count_both_missing / 159737) * 100

# Print the count of rows where both 'x_left_hand_0' and 'x_right_hand_0' have no values and the percentage
print("Number of rows with missing values for both left hand and right hand out of 159737 rows:", count_both_missing)
print("Percentage of rows with missing values for both 'left hand and right hand: {:.2f}%".format(percentage_both_missing))


Number of rows with missing values for both left hand and right hand out of 159737 rows: 65524
Percentage of rows with missing values for both 'left hand and right hand: 41.02%


### Now let's see the average number of frame we have. 

In [33]:

# To find the maximum value for each 'sequence_id':
max_values_per_sequence = test_df.groupby(level='sequence_id').max()

# To display the result:
print(max_values_per_sequence)


             frame  x_right_hand_0  x_left_hand_0  x_pose_0
sequence_id                                                
1880607618     203        0.358531            NaN  0.526337
1880621984      85        0.341914            NaN  0.557909
1880638439     205        0.357306            NaN  0.551323
1880671242     192        0.612402            NaN  0.628354
1880690541     121             NaN       0.723428  0.226055
...            ...             ...            ...       ...
1912534365      76             NaN       0.785545  0.424770
1912545249     124        0.358662            NaN  0.707990
1912581632     172        0.363829            NaN  0.789940
1912612641      96        0.351953            NaN  0.776759
1912627838      93        0.345544            NaN  0.569577

[1000 rows x 4 columns]


In [35]:
# To find the maximum value for each 'sequence_id':
max_values_per_sequence = test_df.groupby(level='sequence_id').max()

# To calculate the average of the maximum values:
average_max_value = max_values_per_sequence.mean()

# To display the result:
print("Average of the maximum values:", average_max_value)

Average of the maximum values: frame             159.205000
x_right_hand_0      0.358610
x_left_hand_0       0.743141
x_pose_0            0.613111
dtype: float64


Here the average of frame is 159(+1) indeed and not 100 what we would see from the describe(). To find the 159(+1) we actually could have also do 159737 / 1000 as the number of total rows and 1000 as the number of individual clip we figured out previously.

Let's see the maximum number of frame. (ps : we can't look at the other stats as biased from the fact every clip start from 0)

In [27]:
test_df['frame'].describe()

count    159737.000000
mean        100.526434
std          77.515498
min           0.000000
25%          41.000000
50%          84.000000
75%         142.000000
max         477.000000
Name: frame, dtype: float64

Next step, understand better those no value, how distribuate are they per clip.

### `Findings : cf to the top of the document.`

### `Summary : there is a large amount of data which is rather clean already. We will want to do some feature reduction, potentially add angles features. A large amount of missing data are present and will be one of the challenge to take into account further in the line during preprocessing. `

<!-- This is at the bottom of the file -->
<a id="bottom"></a>